In [ ]:
import torch 
from pathlib import Path 

In [ ]:
model_name = '../checkpoints/ResNet_bert_energyplus_comstock_keyvalue'
seeds = [1000, 1001, 1002]

models_last = [f'{model_name}_seed={seed}_last.pt' for seed in seeds]
models_best = [f'{model_name}_seed={seed}_best.pt' for seed in seeds]
models_merged = [f'{model_name}_seed={seed}_merged.pt' for seed in seeds]

for m1, m2, mout in zip(models_last, models_best, models_merged):
    
    all_ckpts = [ torch.load( Path(m1) )['model'], torch.load( Path(m2) )['model'] ]

    new_state_dict = {'model': {}}
    # for each parameter
    for k in all_ckpts[0].keys():
        agg= []
        # aggregate across all checkpoints
        for ckpt in all_ckpts:
            agg += [ckpt[k]]
        
        #print(agg)
        try:
            new_state_dict['model'][k] = torch.mean(torch.stack(agg,0), dim=0)
        except RuntimeError:
            if 'l_kernel' in k:
                new_state_dict['model'][k] = agg[0]
                print(k, agg)
        #print(new_state_dict['model'][k])

    torch.save(new_state_dict, Path(mout) )